In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
--2018-10-31 07:27:42--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.222, 91.189.89.223
Connecting to launchpad.net (launchpad.net)|91.189.89.222|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb [following]
--2018-10-31 07:27:44--  https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpadlibrarian.net (launchpadlibrarian.net)... 91.189.89.228, 91.189.89.229
Connecting to launchpadlibrarian.net (launchpadlibrarian.net)|91.189.89.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1232624 (1.2M) [application/x-debian-package]
Saving to: ‘google-drive-ocamlfuse_

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [3]:
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [0]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [5]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('drive/deeplearning/pokeman_dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('drive/deeplearning/pokeman_dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 445 images belonging to 3 classes.
Found 238 images belonging to 3 classes.


In [0]:
from keras.applications import VGG16

In [9]:
conv_base = VGG16(weights='imagenet',
include_top=False,
input_shape=(150, 150, 3))

58892288/58889256 [==============================] - 6s 0us/step


In [0]:
from keras import models
from keras import layers
import os
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [0]:
conv_base.trainable = False #do not train the conv-base

In [0]:
base_dir = 'drive/deeplearning/pokeman_dataset'
train_dir = os.path.join(base_dir, 'training_set')

test_dir = os.path.join(base_dir, 'test_set')

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='categorical')

Found 445 images belonging to 3 classes.


In [15]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=20,
                                                  class_mode='categorical')

Found 238 images belonging to 3 classes.


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

In [17]:
history = model.fit_generator(train_generator, steps_per_epoch=100,
                              epochs=30,
                              validation_data=test_generator,
                              validation_steps=50)

Epoch 1/30
  8/100 [=>............................] - ETA: 1:45 - loss: 1.0339 - acc: 0.4313

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 162s 2s/step - loss: 0.6840 - acc: 0.7640 - val_loss: 0.4632 - val_acc: 0.8770
Epoch 2/30
100/100 [==============================] - 122s 1s/step - loss: 0.3641 - acc: 0.8960 - val_loss: 0.3301 - val_acc: 0.8931
Epoch 3/30
100/100 [==============================] - 127s 1s/step - loss: 0.2608 - acc: 0.9230 - val_loss: 0.2874 - val_acc: 0.8841
Epoch 4/30
100/100 [==============================] - 125s 1s/step - loss: 0.2142 - acc: 0.9440 - val_loss: 0.2484 - val_acc: 0.9254
Epoch 5/30
100/100 [==============================] - 121s 1s/step - loss: 0.1749 - acc: 0.9450 - val_loss: 0.2259 - val_acc: 0.9083
Epoch 6/30
100/100 [==============================] - 129s 1s/step - loss: 0.1643 - acc: 0.9490 - val_loss: 0.2083 - val_acc: 0.9173
Epoch 7/30
100/100 [==============================] - 125s 1s/step - loss: 0.1352 - acc: 0.9650 - val_loss: 0.1916 - val_acc: 0.9224
Epoch 8/30
100/100 [==============================] - 121s 1s/step - loss: 0.133

In [0]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
    
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [20]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=test_generator,
    validation_steps=50)

Epoch 1/10
 18/100 [====>.........................] - ETA: 1:20 - loss: 0.0201 - acc: 0.9944

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 137s 1s/step - loss: 0.0170 - acc: 0.9953 - val_loss: 0.0771 - val_acc: 0.9724
Epoch 2/10
100/100 [==============================] - 125s 1s/step - loss: 0.0096 - acc: 0.9973 - val_loss: 0.1514 - val_acc: 0.9536
Epoch 3/10
100/100 [==============================] - 130s 1s/step - loss: 0.0039 - acc: 0.9990 - val_loss: 0.0553 - val_acc: 0.9859
Epoch 4/10
100/100 [==============================] - 128s 1s/step - loss: 0.0058 - acc: 0.9970 - val_loss: 0.0893 - val_acc: 0.9691
Epoch 5/10
100/100 [==============================] - 123s 1s/step - loss: 0.0079 - acc: 0.9973 - val_loss: 0.0833 - val_acc: 0.9637
Epoch 6/10
100/100 [==============================] - 132s 1s/step - loss: 0.0033 - acc: 0.9993 - val_loss: 0.0549 - val_acc: 0.9785
Epoch 7/10
100/100 [==============================] - 128s 1s/step - loss: 0.0015 - acc: 0.9993 - val_loss: 0.0788 - val_acc: 0.9751
Epoch 8/10
100/100 [==============================] - 126s 1s/step - loss: 0.001

In [23]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('drive/deeplearning/pokeman_dataset/single_prediction/pokemon1.png', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

{'bulbasaur': 0, 'charmander': 1, 'pikachu': 2}

In [24]:
if int(result[0][0]) == 1:
    prediction = 'Bulbasaur'
elif int(result[0][1]) == 1:
    prediction = 'Charmander'
else:
    prediction = 'Pikachu'

print (prediction)

Bulbasaur


In [26]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('drive/deeplearning/pokeman_dataset/single_prediction/pokemon2.png', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
if int(result[0][0]) == 1:
    prediction = 'Bulbasaur'
elif int(result[0][1]) == 1:
    prediction = 'Charmander'
else:
    prediction = 'Pikachu'

print (prediction)

Charmander


In [27]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('drive/deeplearning/pokeman_dataset/single_prediction/pokemon3.png', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
if int(result[0][0]) == 1:
    prediction = 'Bulbasaur'
elif int(result[0][1]) == 1:
    prediction = 'Charmander'
else:
    prediction = 'Pikachu'

print (prediction)

Pikachu
